In [9]:
import pandas as pd
import datetime as dt
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split

In [68]:
def classification_(x,y):
    
    g=GaussianNB()
    b=BernoulliNB()
    k=KNeighborsClassifier()
    svc=SVC()
    d=DecisionTreeClassifier()
    log=LogisticRegression()
    gbc=GradientBoostingClassifier()
    mn=MultinomialNB()
    rf=RandomForestClassifier()
    ab=AdaBoostClassifier()
    
    x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=.20, random_state=42)
    
    algos=[g, b, k, svc, d, log, gbc, mn, rf, ab]
    algos_name=['Gaussian', 'Bernoulli', 'KNEighbors', 'SVC', 'DecisionTree', 'LogisticRegression', 'GradientBoosting', 'Multinomial', 'RandomForest', 'AdaBoost']
    
    accuracy = []
    precision = []
    recall = []
    f1 = []
    
    result=pd.DataFrame(columns=['AccuracyScore', 'PrecisionScore', 'RecallScore', 'f1_Score'], index=algos_name)
    
    for i in algos:
        
        rus=RandomUnderSampler(random_state=42)
        
        x_rus,y_rus=rus.fit_resample(x_train,y_train)
        
        predict=i.fit(x_rus,y_rus).predict(x_test)
        
        accuracy.append(accuracy_score(y_test,predict))
        
        precision.append(precision_score(y_test,predict))
        
        recall.append(recall_score(y_test,predict))
        
        f1.append(f1_score(y_test,predict))
        
    result.AccuracyScore=accuracy
    result.PrecisionScore=precision
    result.RecallScore=recall
    result.f1_Score=f1
    
    return result.sort_values('AccuracyScore', ascending=False)

In [15]:
df=pd.read_csv("failure.csv")


In [16]:
df.shape

(124494, 12)

In [20]:
df.duplicated().sum()
#tekrar eden veri yok

0

In [21]:
df.sample(10)

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
86500,2015-05-28,W1F0L2Q7,0,52483848,0,0,0,61,245895,0,0,0
3655,2015-01-04,S1F0QWYG,0,239546568,0,0,0,6,197234,0,0,0
105716,2015-07-23,S1F0FP0C,0,105732696,0,0,0,10,283263,0,0,0
103598,2015-07-16,W1F1MRP9,0,16653288,0,0,0,6,241353,0,0,0
89666,2015-06-06,W1F14FS4,0,241221560,0,0,0,6,258191,0,0,0
117619,2015-09-04,Z1F0LT3Y,0,83329408,0,0,0,10,494417,0,0,0
86942,2015-05-29,Z1F0L3BL,0,28515016,0,0,0,6,320534,0,0,0
5990,2015-01-06,S1F0R2VR,0,61724120,0,0,0,6,192745,0,0,0
41700,2015-02-24,W1F1FHEK,0,233333824,0,0,0,13,44,0,0,0
96656,2015-06-26,W1F14FS4,0,56118824,0,0,0,6,258214,0,0,0


In [24]:
df.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        124494 non-null  object
 1   device      124494 non-null  object
 2   failure     124494 non-null  int64 
 3   attribute1  124494 non-null  int64 
 4   attribute2  124494 non-null  int64 
 5   attribute3  124494 non-null  int64 
 6   attribute4  124494 non-null  int64 
 7   attribute5  124494 non-null  int64 
 8   attribute6  124494 non-null  int64 
 9   attribute7  124494 non-null  int64 
 10  attribute8  124494 non-null  int64 
 11  attribute9  124494 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 11.4+ MB


In [25]:
df.isnull().sum()        
#boş verim yok

date          0
device        0
failure       0
attribute1    0
attribute2    0
attribute3    0
attribute4    0
attribute5    0
attribute6    0
attribute7    0
attribute8    0
attribute9    0
dtype: int64

In [27]:
#date sütununu gün ay yıl şeklinde ayrı sütunlar olarak ayıralım------>

In [29]:
df['date']=pd.to_datetime(df['date'])

In [30]:
df['day']=df['date'].dt.day
df['month']=df['date'].dt.month
df['year']=df['date'].dt.year


In [31]:
df

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,day,month,year
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7,1,1,2015
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0,1,1,2015
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0,1,1,2015
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0,1,1,2015
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3,1,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124489,2015-11-02,Z1F0MA1S,0,18310224,0,0,0,10,353705,8,8,0,2,11,2015
124490,2015-11-02,Z1F0Q8RT,0,172556680,96,107,4,11,332792,0,0,13,2,11,2015
124491,2015-11-02,Z1F0QK05,0,19029120,4832,0,0,11,350410,0,0,0,2,11,2015
124492,2015-11-02,Z1F0QL3N,0,226953408,0,0,0,12,358980,0,0,0,2,11,2015


In [32]:
df.failure.value_counts()

# 106 kez hata vermiş

0    124388
1       106
Name: failure, dtype: int64

In [33]:
df.nunique()

date             304
device          1168
failure            2
attribute1    123878
attribute2       558
attribute3        47
attribute4       115
attribute5        60
attribute6     44838
attribute7        28
attribute8        28
attribute9        65
day               31
month             11
year               1
dtype: int64

In [34]:
df.describe()

,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,day,month,year
count,124494.000000,1.244940e+05,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.0
mean,0.000851,1.223868e+08,159.484762,9.940455,1.741120,14.222693,260172.858025,0.292528,0.292528,12.451524,14.894196,4.028299,2015.0
std,0.029167,7.045960e+07,2179.657730,185.747321,22.908507,15.943021,99151.009852,7.436924,7.436924,191.425623,8.759542,2.572167,0.0
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2015.0
25%,0.000000,6.127675e+07,0.000000,0.000000,0.000000,8.000000,221452.000000,0.000000,0.000000,0.000000,7.000000,2.000000,2015.0
50%,0.000000,1.227957e+08,0.000000,0.000000,0.000000,10.000000,249799.500000,0.000000,0.000000,0.000000,15.000000,3.000000,2015.0
75%,0.000000,1.833084e+08,0.000000,0.000000,0.000000,12.000000,310266.000000,0.000000,0.000000,0.000000,22.000000,6.000000,2015.0
max,1.000000,2.441405e+08,64968.000000,24929.000000,1666.000000,98.000000,689161.000000,832.000000,832.000000,18701.000000,31.000000,11.000000,2015.0


In [35]:
df.corr()

,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,day,month,year
failure,1.000000,0.001984,0.052902,-0.000948,0.067398,0.002270,-0.000550,0.119055,0.119055,0.001622,0.002994,0.000321,NaN
attribute1,0.001984,1.000000,-0.004248,0.003702,0.001837,-0.003370,-0.001516,0.000151,0.000151,0.001122,-0.000239,-0.005613,NaN
attribute2,0.052902,-0.004248,1.000000,-0.002617,0.146593,-0.013999,-0.026350,0.141367,0.141367,-0.002736,-0.010000,-0.006416,NaN
attribute3,-0.000948,0.003702,-0.002617,1.000000,0.097452,-0.006696,0.009027,-0.001884,-0.001884,0.532366,-0.004232,-0.001101,NaN
attribute4,0.067398,0.001837,0.146593,0.097452,1.000000,-0.009773,0.024870,0.045631,0.045631,0.036069,-0.008383,-0.005490,NaN
attribute5,0.002270,-0.003370,-0.013999,-0.006696,-0.009773,1.000000,-0.017051,-0.009384,-0.009384,0.005949,0.009526,-0.014129,NaN
attribute6,-0.000550,-0.001516,-0.026350,0.009027,0.024870,-0.017051,1.000000,-0.012207,-0.012207,0.021152,0.019483,0.138159,NaN
attribute7,0.119055,0.000151,0.141367,-0.001884,0.045631,-0.009384,-0.012207,1.000000,1.000000,0.006861,0.009194,0.020576,NaN
attribute8,0.119055,0.000151,0.141367,-0.001884,0.045631,-0.009384,-0.012207,1.000000,1.000000,0.006861,0.009194,0.020576,NaN
attribute9,0.001622,0.001122,-0.002736,0.532366,0.036069,0.005949,0.021152,0.006861,0.006861,1.000000,-0.012316,-0.025383,NaN


In [36]:
#corr ve failure eşit etkiliyor

In [37]:
df.corr()['failure'].sort_values(ascending=False)
#7 ve 8 aynı, birini silelim--->

failure       1.000000
attribute7    0.119055
attribute8    0.119055
attribute4    0.067398
attribute2    0.052902
day           0.002994
attribute5    0.002270
attribute1    0.001984
attribute9    0.001622
month         0.000321
attribute6   -0.000550
attribute3   -0.000948
year               NaN
Name: failure, dtype: float64

In [38]:
del df['attribute8']

In [39]:
df=df.drop('date', axis=1)

In [40]:
df[['attribute3', 'attribute4', 'attribute7', 'attribute9']]=df[['attribute3', 'attribute4', 'attribute7', 'attribute9']].astype('object')

In [41]:
df.info()

#3,4,7,9 object yaptık

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   device      124494 non-null  object
 1   failure     124494 non-null  int64 
 2   attribute1  124494 non-null  int64 
 3   attribute2  124494 non-null  int64 
 4   attribute3  124494 non-null  object
 5   attribute4  124494 non-null  object
 6   attribute5  124494 non-null  int64 
 7   attribute6  124494 non-null  int64 
 8   attribute7  124494 non-null  object
 9   attribute9  124494 non-null  object
 10  day         124494 non-null  int64 
 11  month       124494 non-null  int64 
 12  year        124494 non-null  int64 
dtypes: int64(8), object(5)
memory usage: 12.3+ MB


In [42]:
df['device']=[i[:4] for i in df.device.values.tolist()]

In [44]:
df
#device sütunundaki isimlerin ilk 4 harfini aldık

,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute9,day,month,year
0,S1F0,0,215630672,56,0,52,6,407438,0,7,1,1,2015
1,S1F0,0,61370680,0,3,0,6,403174,0,0,1,1,2015
2,S1F0,0,173295968,0,0,0,12,237394,0,0,1,1,2015
3,S1F0,0,79694024,0,0,0,6,410186,0,0,1,1,2015
4,S1F0,0,135970480,0,0,0,15,313173,0,3,1,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124489,Z1F0,0,18310224,0,0,0,10,353705,8,0,2,11,2015
124490,Z1F0,0,172556680,96,107,4,11,332792,0,13,2,11,2015
124491,Z1F0,0,19029120,4832,0,0,11,350410,0,0,2,11,2015
124492,Z1F0,0,226953408,0,0,0,12,358980,0,0,2,11,2015


In [45]:
df.device.value_counts()

S1F0    33169
W1F0    23294
S1F1    21689
W1F1    19974
Z1F0    18869
Z1F1     7248
Z1F2      251
Name: device, dtype: int64

In [48]:
df.device.describe()

count     124494
unique         7
top         S1F0
freq       33169
Name: device, dtype: object

In [50]:
outliers=df.quantile(.95)
df=df[(df['attribute1']<outliers['attribute1'])]
df=df[(df['attribute2']<outliers['attribute2'])]
df=df[(df['attribute6']<outliers['attribute6'])]

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106351 entries, 1 to 124493
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   device      106351 non-null  object
 1   failure     106351 non-null  int64 
 2   attribute1  106351 non-null  int64 
 3   attribute2  106351 non-null  int64 
 4   attribute3  106351 non-null  object
 5   attribute4  106351 non-null  object
 6   attribute5  106351 non-null  int64 
 7   attribute6  106351 non-null  int64 
 8   attribute7  106351 non-null  object
 9   attribute9  106351 non-null  object
 10  day         106351 non-null  int64 
 11  month       106351 non-null  int64 
 12  year        106351 non-null  int64 
dtypes: int64(8), object(5)
memory usage: 11.4+ MB


In [52]:
x=df.drop('failure', axis=1)
y=df['failure']

In [53]:
x=pd.get_dummies(x,drop_first=True)

In [54]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import geometric_mean_score, make_index_balanced_accuracy, classification_report_imbalanced
from sklearn.metrics import confusion_matrix

In [55]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)
gbc=GradientBoostingClassifier()

In [66]:
rus=RandomUnderSampler(random_state=42)
x_rus,y_rus=rus.fit_resample(x_train,y_train)
pred=gbc.fit(x_rus,y_rus).predict(x_test)

In [65]:
confusion_matrix(y_test,pred)

array([[15768, 10803],
       [    4,    13]], dtype=int64)

In [69]:
classification_(x,y)

,AccuracyScore,PrecisionScore,RecallScore,f1_Score
Bernoulli,0.951577,0.002933,0.230769,0.005792
RandomForest,0.747450,0.001488,0.615385,0.002970
AdaBoost,0.551831,0.001153,0.846154,0.002302
GradientBoosting,0.550656,0.001150,0.846154,0.002296
DecisionTree,0.543275,0.001029,0.769231,0.002054
Multinomial,0.532744,0.000704,0.538462,0.001407
KNEighbors,0.498378,0.000843,0.692308,0.001684
SVC,0.374924,0.000602,0.615385,0.001202
Gaussian,0.316064,0.000756,0.846154,0.001510
LogisticRegression,0.305533,0.000677,0.769231,0.001352


In [72]:
import tensorflow as tf
model=tf.keras.Sequential([
                           tf.keras.layers.Dense(512, activation='relu'),
                           tf.keras.layers.Dense(512, activation='relu'),
                           tf.keras.layers.Dropout(0.25),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(512, activation='relu'),
                           tf.keras.layers.Dropout(0.5),
                           tf.keras.layers.Dense(512, activation='relu'),
                           tf.keras.layers.Dense(10, activation='softmax'),])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history=model.fit(x_rus,y_rus, batch_size=32, epochs=10, verbose=1, validation_data=(x_test,y_test))

Epoch 1/10
3/3 [==============================] - 18s 6s/step - loss: 5174169.5000 - accuracy: 0.2727 - val_loss: 774407.1875 - val_accuracy: 0.0031
Epoch 2/10
3/3 [==============================] - 11s 5s/step - loss: 2221366.2500 - accuracy: 0.5455 - val_loss: 1037.8566 - val_accuracy: 0.9994
Epoch 3/10
3/3 [==============================] - 11s 6s/step - loss: 2491013.2500 - accuracy: 0.4242 - val_loss: 5812625.0000 - val_accuracy: 6.3939e-04
Epoch 4/10
3/3 [==============================] - 11s 6s/step - loss: 2347789.2500 - accuracy: 0.5000 - val_loss: 1103.6648 - val_accuracy: 0.9994
Epoch 5/10
3/3 [==============================] - 11s 6s/step - loss: 1224751.0000 - accuracy: 0.5303 - val_loss: 143956.7656 - val_accuracy: 0.0012
Epoch 6/10
3/3 [==============================] - 11s 6s/step - loss: 2077002.8750 - accuracy: 0.4545 - val_loss: 3923341.5000 - val_accuracy: 6.3939e-04
Epoch 7/10
3/3 [==============================] - 12s 6s/step - loss: 2231753.2500 - accuracy: 0.454

In [73]:
_, accuracy=model.evaluate(x_test, y_test)
print('Accuracy: %2f' % (accuracy*100))

831/831 [==============================] - 15s 18ms/step - loss: 1758.8545 - accuracy: 0.9994
Accuracy: 99.936062


In [74]:
#DL ile başarı oranım: 99.93